In [ ]:
import numpy as np
import networkx as nx
#import tensorflow as tf
import math

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
#from tensorflow.keras import Sequential, Model
#from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Embedding
from matplotlib.lines import Line2D
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
#matplotlib.rcParams['text.usetex'] = True

In [ ]:
def read_results_txt_file(filename, n_a, n_b):
        #c_a = 'red'
        #c_b = 'blue'
        c_a = '1'
        c_b = '0'
        #n_a = np.sum([1 if v['color'] == c_a else 0 for _, v in self.G.nodes.items()])
        #n_b = np.sum([1 if v['color'] == c_b else 0 for _, v in self.G.nodes.items()])

        with open(filename, 'r') as f:
            inf_a, inf_b = [], []
            for line in f:
                s = line.split()
                if s[1] == c_a and s[3] == c_b:
                    inf_a.append(float(s[2]))
                    inf_b.append(float(s[4]))
                elif s[1] == c_b and s[3] == c_a:
                    inf_b.append(float(s[2]))
                    inf_a.append(float(s[4]))
                else:
                    raise Exception('wrong format')

        inf_a, inf_b = np.array(inf_a), np.array(inf_b)
        total_fraction = 100 * (inf_a + inf_b) / (n_a + n_b)
        #diff_fraction = np.abs( (100 * inf_a / n_a) - (100 * inf_b / n_b) )
        var_fraction = np.var(np.concatenate([(100 * inf_a / n_a).reshape([-1,1]) , 
                                              (100 * inf_b / n_b).reshape([-1,1])], axis=1), axis=1)

        frac_a = 100 * inf_a / n_a
        frac_b = 100 * inf_b / n_b
    
        return np.concatenate([np.array(total_fraction).reshape([-1,1]),
                np.array(frac_a).reshape([-1,1]),
                np.array(frac_b).reshape([-1,1]),
                #np.array(diff_fraction).reshape([-1,1])], axis=1)
                np.array(var_fraction).reshape([-1,1])], axis=1)


def read_avg_results_txt_file(filename, n_a, n_b):
    res = []
    for i in ['_1', '_2', '_3', '_4', '_5']:
        res.append(read_results_txt_file(filename + i + '_results.txt', n_a, n_b))

    print(res[0].shape, ' ', res[1].shape)
        
    adv = np.mean( np.concatenate([np.expand_dims(a, 2) for a in res], axis = 2), axis = 2)
    return adv

In [ ]:
n_a = 344
n_b = 97

bndry = '0.5'
exp_ = '4.0'

# filename = '../influence_maximization/results/rice/rice_subset_emb_random_walk_5_bndry_' + bndry + '_exp_' + exp_ + '_d32_results.txt'
# filename = '../influence_maximization/results/rice/rice_subset_emb_randomembedding_d32'
filename = '../influence_maximization/results/rice/rice_subset_emb_random_walk_5_bndry_' + bndry + '_exp_' + exp_ + '_d32'
rice_rw_method = read_avg_results_txt_file(filename, n_a, n_b)
# rice_rw_method = read_results_txt_file(filename, n_a, n_b)

filename = '../influence_maximization/results/rice/rice_subset_greedy__results.txt'
rice_greedy = read_results_txt_file(filename, n_a, n_b)

# filename = '../influence_maximization/results/rice/rice_subset_emb_unweighted_d32_results.txt'
# rice_unweighted = read_results_txt_file(filename, n_a, n_b)
filename = '../influence_maximization/results/rice/rice_subset_emb_unweighted_d32'
rice_unweighted = read_avg_results_txt_file(filename, n_a, n_b)


filename = '../influence_maximization/results/rice/rice_subset_emb_random_walk_5_bndry_' + '0.5' + '_exp_' + '0.0' + '_d32'
fairwalk = read_avg_results_txt_file(filename, n_a, n_b)

###################

uw = np.array([0.0923, rice_unweighted[-1,0], rice_unweighted[-1,3]])

filename = '../influence_maximization/results/rice/rice_subset_emb_random_walk_5_bndry_' + '0.3' + '_exp_' + '4.0' + '_d32'
tmp = read_avg_results_txt_file(filename, n_a, n_b)
alpha0_3_p_4 = np.array([0.642, tmp[-1,0], tmp[-1,3]])

filename = '../influence_maximization/results/rice/rice_subset_emb_random_walk_5_bndry_' + '0.5' + '_exp_' + '4.0' + '_d32'
tmp = read_avg_results_txt_file(filename, n_a, n_b)
alpha0_5_p_4 = np.array([0.0173, tmp[-1,0], tmp[-1,3]])

filename = '../influence_maximization/results/rice/rice_subset_emb_random_walk_5_bndry_' + '0.7' + '_exp_' + '4.0' + '_d32'
tmp = read_avg_results_txt_file(filename, n_a, n_b)
alpha0_7_p_4 = np.array([0.011, tmp[-1,0], tmp[-1,3]])

filename = '../influence_maximization/results/rice/rice_subset_emb_randomembedding_d32'
tmp = read_avg_results_txt_file(filename, n_a, n_b)
rnd = np.array([0.011, tmp[-1,0], tmp[-1,3]])

In [ ]:
print(rice_rw_method[-1,0], rice_rw_method[-1,3])

In [ ]:
adv_1 = np.array([
    [0.014, 0.024, 0.01, 0.014],
    [0.036, 0.031, 0.038, 0.006999999999999999],
    [0.051, 0.045, 0.053, 0.008],
    [0.066, 0.07, 0.065, 0.0050000000000000044],
    [0.077, 0.081, 0.076, 0.0050000000000000044],
    [0.096, 0.077, 0.102, 0.024999999999999994],
    [0.109, 0.101, 0.111, 0.009999999999999995],
    [0.125, 0.106, 0.131, 0.02500000000000001],
    [0.139, 0.121, 0.144, 0.022999999999999993],
    [0.152, 0.122, 0.161, 0.03900000000000001]
])

adv_2 = np.array([
    [0.014, 0.024, 0.012, 0.012] ,
   [0.035, 0.041, 0.033, 0.008] ,
   [0.05, 0.056, 0.048, 0.008] ,
   [0.07, 0.083, 0.066, 0.017] ,
   [0.083, 0.107, 0.077, 0.03] ,
   [0.104, 0.113, 0.102, 0.01100000000000001] ,
   [0.116, 0.126, 0.114, 0.011999999999999997] ,
   [0.13, 0.149, 0.124, 0.024999999999999994] ,
   [0.144, 0.151, 0.142, 0.009000000000000008] ,
   [0.158, 0.165, 0.156, 0.009000000000000008]
])
adv_3 = np.array([
    [0.019, 0.017, 0.019, 0.0019999999999999983] ,
   [0.033, 0.03, 0.035, 0.0050000000000000044] ,
   [0.049, 0.067, 0.044, 0.023000000000000007] ,
   [0.066, 0.059, 0.068, 0.009000000000000008] ,
   [0.083, 0.084, 0.082, 0.0020000000000000018] ,
   [0.098, 0.099, 0.098, 0.0010000000000000009] ,
   [0.11, 0.122, 0.107, 0.015] ,
   [0.132, 0.108, 0.139, 0.031000000000000014] ,
   [0.146, 0.121, 0.153, 0.032] ,
   [0.155, 0.164, 0.153, 0.01100000000000001]
])
adv_4 = np.array([
    [0.018, 0.005, 0.021, 0.016] ,
   [0.035, 0.02, 0.039, 0.019] ,
   [0.051, 0.024, 0.059, 0.034999999999999996] ,
   [0.07, 0.028, 0.082, 0.054000000000000006] ,
   [0.087, 0.054, 0.096, 0.042] ,
   [0.102, 0.059, 0.114, 0.05500000000000001] ,
   [0.119, 0.064, 0.134, 0.07] ,
   [0.133, 0.067, 0.152, 0.08499999999999999] ,
   [0.147, 0.091, 0.163, 0.07200000000000001] ,
   [0.164, 0.096, 0.183, 0.087]
])
adv_5 = np.array([
    [0.017, 0.015, 0.018, 0.002999999999999999] ,
   [0.034, 0.017, 0.038, 0.020999999999999998] ,
   [0.051, 0.044, 0.053, 0.009000000000000001] ,
   [0.067, 0.038, 0.075, 0.037] ,
   [0.085, 0.054, 0.094, 0.04] ,
   [0.103, 0.068, 0.112, 0.044] ,
   [0.113, 0.09, 0.12, 0.03] ,
   [0.134, 0.086, 0.147, 0.061] ,
   [0.141, 0.097, 0.154, 0.056999999999999995] ,
   [0.163, 0.106, 0.179, 0.073]
])
adv_6 = np.array([
    [0.017, 0.004, 0.021, 0.017] ,
   [0.035, 0.02, 0.04, 0.02] ,
   [0.053, 0.036, 0.058, 0.022000000000000006] ,
   [0.066, 0.037, 0.075, 0.038] ,
   [0.081, 0.063, 0.086, 0.022999999999999993] ,
   [0.095, 0.086, 0.097, 0.01100000000000001] ,
   [0.107, 0.12, 0.103, 0.017] ,
   [0.125, 0.115, 0.128, 0.012999999999999998] ,
   [0.141, 0.13, 0.144, 0.013999999999999985] ,
   [0.155, 0.144, 0.158, 0.014000000000000012]
])
adv_7 = np.array([
    [0.022, 0.005, 0.027, 0.022] ,
   [0.036, 0.031, 0.037, 0.005999999999999998] ,
   [0.051, 0.046, 0.053, 0.006999999999999999] ,
   [0.068, 0.061, 0.07, 0.009000000000000008] ,
   [0.081, 0.074, 0.083, 0.009000000000000008] ,
   [0.096, 0.077, 0.102, 0.024999999999999994] ,
   [0.112, 0.113, 0.111, 0.0020000000000000018] ,
   [0.128, 0.128, 0.128, 0.0] ,
   [0.139, 0.141, 0.138, 0.002999999999999975] ,
   [0.159, 0.148, 0.163, 0.015000000000000013]
])
adv_8 = np.array([
    [0.014, 0.025, 0.011, 0.014000000000000002] ,
   [0.029, 0.028, 0.029, 0.0010000000000000009] ,
   [0.045, 0.032, 0.049, 0.017] ,
   [0.061, 0.046, 0.065, 0.019000000000000003] ,
   [0.081, 0.051, 0.089, 0.038] ,
   [0.093, 0.053, 0.105, 0.052] ,
   [0.11, 0.067, 0.123, 0.055999999999999994] ,
   [0.125, 0.082, 0.137, 0.05500000000000001] ,
   [0.143, 0.109, 0.152, 0.043] ,
   [0.157, 0.113, 0.169, 0.05600000000000001]
])
adv_9 = np.array([
    [0.018, 0.004, 0.022, 0.018] ,
   [0.036, 0.02, 0.04, 0.02] ,
   [0.053, 0.034, 0.059, 0.024999999999999994] ,
   [0.063, 0.056, 0.065, 0.009000000000000001] ,
   [0.084, 0.086, 0.084, 0.001999999999999988] ,
   [0.095, 0.098, 0.095, 0.0030000000000000027] ,
   [0.111, 0.09, 0.117, 0.02700000000000001] ,
   [0.13, 0.097, 0.139, 0.04200000000000001] ,
   [0.145, 0.122, 0.151, 0.028999999999999998] ,
   [0.157, 0.111, 0.17, 0.05900000000000001]
])
adv_10 = np.array([
    [0.013, 0.025, 0.01, 0.015000000000000001] ,
   [0.037, 0.02, 0.041, 0.021] ,
   [0.057, 0.038, 0.063, 0.025] ,
   [0.069, 0.061, 0.071, 0.009999999999999995] ,
   [0.08, 0.062, 0.085, 0.023000000000000007] ,
   [0.102, 0.078, 0.109, 0.031] ,
   [0.109, 0.11, 0.108, 0.0020000000000000018] ,
   [0.134, 0.117, 0.138, 0.021000000000000005] ,
   [0.143, 0.13, 0.147, 0.016999999999999987] ,
   [0.155, 0.14, 0.159, 0.01899999999999999]
])




# adv = np.mean( np.concatenate([np.expand_dims(a, 2) for a in 
#             [adv_1, adv_2, adv_3, adv_4, adv_5]], axis = 2), axis = 2)

tmp = np.concatenate([np.expand_dims(100 * a, 2) for a in [adv_1, adv_2, adv_3, adv_4, adv_5]], axis = 2)
for i in range(tmp.shape[2]):
    tmp[:,3,i] = np.var(tmp[:,1:3,i], axis=1)
adv = np.mean(tmp, axis=2)

In [ ]:
n_seeds = np.arange(2,41,2)

red_ = '#fab3ac'
blue_ = '#29a5e3'
cyan_ = '#d2f0f7'
green_ = '#a3f77e'
gray_ = '#dbdbdb'
purple_ = '#BF55EC'
yellow_ = '#F7CA18'

In [ ]:
def get_real_plots(figtype, square, fontsize, labelsize, imagesize, line_width, ylim=None, legend_size=20):

    fig, ax = plt.subplots()

    
    ind = n_seeds - 1
    if figtype == 'total':
        ax.plot(n_seeds, rice_unweighted[ind, 0], color='red', lw=line_width, label='DeepWalk')
    elif figtype == 'frac':
        ax.plot(n_seeds, rice_unweighted[ind, 1], color='red', lw=line_width, linestyle='dashed')
        ax.plot(n_seeds, rice_unweighted[ind, 2],  color='red', lw=line_width, linestyle='dotted')
        
    if figtype == 'total':
        ax.plot(n_seeds, fairwalk[ind, 0], color='green', lw=line_width, label='FairWalk')
    elif figtype == 'frac':
        ax.plot(n_seeds, fairwalk[ind, 1], color='green', lw=line_width, linestyle='dashed')
        ax.plot(n_seeds, fairwalk[ind, 2],  color='green', lw=line_width, linestyle='dotted')
        
    #n_seeds_adv = np.concatenate([np.arange(4,33,4), np.array([40])])
    n_seeds_adv = np.arange(4,41,4)
    if figtype == 'total':
        ax.plot(n_seeds_adv, adv[:, 0], color='cyan', lw=line_width, label='Adversarial')
    elif figtype == 'frac':
        ax.plot(n_seeds_adv, adv[:, 2], color='cyan', lw=line_width, linestyle='dashed')
        ax.plot(n_seeds_adv, adv[:, 1],  color='cyan', lw=line_width, linestyle='dotted')
    
    if figtype == 'total':
        ax.plot(n_seeds, rice_rw_method[ind, 0], color='blue', lw=line_width, label='CrossWalk')
    elif figtype == 'frac':
        ax.plot(n_seeds, rice_rw_method[ind, 1], color='blue', lw=line_width, linestyle='dashed')
        ax.plot(n_seeds, rice_rw_method[ind, 2],  color='blue', lw=line_width, linestyle='dotted')
    
    if figtype == 'total':
        plt.legend(loc='upper left', prop={'size': legend_size})
#     custom_lines = [Line2D([0], [0], color=red_, lw=3),
#                 Line2D([0], [0], color=green_, lw=3), 
#                 Line2D([0], [0], color=cyan_, lw=3),
#                 Line2D([0], [0], color=blue_, lw=3)]
#     legend1 = plt.legend(custom_lines, ['Greedy', 'DeepWalk', 'FairWalk', 'FairWalk++'], bbox_to_anchor=(-0.24, 1.06, 1.4, .102), loc='lower left',
#            ncol=4, mode="expand", borderaxespad=0.)
#     plt.gca().add_artist(legend1)
    
    if figtype == 'frac':
        custom_lines2 = [Line2D([0], [0], linestyle='dashed', lw=3),
                Line2D([0], [0], linestyle='dotted', lw=3)]
        legend2 = plt.legend(custom_lines2, ['Group A', 'Group B'], loc='upper left')
        plt.gca().add_artist(legend2)
    
    if ylim:
        ax.set_ylim([0, ylim])
    
    ax.set_axisbelow(True)
    ax.yaxis.grid(color='gray', linestyle='dashed')
    
    plt.xlabel('Number of Seeds', fontsize = labelsize)
    if figtype == 'total':
        #plt.ylabel('Fraction Influenced', fontsize = labelsize)
        plt.ylabel('Total Influence', fontsize = labelsize)
    else:
        plt.ylabel('Group Influence', fontsize = labelsize)
    plt.rcParams.update({'font.size': fontsize})
    plt.xticks(fontsize=labelsize)
    plt.yticks([5,10,15,20], ['5%', '10%', '15%', '20%'], fontsize=labelsize)
    fig.set_size_inches(imagesize[0], imagesize[1]) 
    
    added_s = ''
    if not square:
        added_s = '1'
    
    if figtype == 'total':
        fig.savefig('fig/influence_maximization_rice/rice-total' + added_s + '.pdf',bbox_inches='tight')
    elif figtype == 'frac':
        fig.savefig('fig/influence_maximization_rice/rice-fractions' + added_s + '.pdf',bbox_inches='tight')
    
    
#%%

# def get_diff_plot(real, square, fontsize, labelsize, imagesize, line_width, ylim=None):
def get_var_plot(real, square, fontsize, labelsize, imagesize, line_width, ylim=None, legend_size=20):
        
    fig, ax = plt.subplots()
    
    ind = n_seeds - 1
    ax.plot(n_seeds, rice_unweighted[ind, 3], color='red', lw=line_width, label='DeepWalk')
    ax.plot(n_seeds, fairwalk[ind, 3], color='green', lw=line_width, label='FairWalk')
#     n_seeds_adv = np.concatenate([np.arange(4,33,4), np.array([40])])
    n_seeds_adv = np.arange(4,41,4)
    ax.plot(n_seeds_adv, adv[:, 3], color='cyan', lw=line_width, label='Adversarial')
    ax.plot(n_seeds, rice_rw_method[ind, 3], color='blue', lw=line_width, label='CrossWalk')
    
#     custom_lines = [Line2D([0], [0], color='red', lw=3),
#                 Line2D([0], [0], color='green', lw=3),
#                 Line2D([0], [0], color='blue', lw=3), 
#                 Line2D([0], [0], color='cyan', lw=3)]
#     legend1 = plt.legend(custom_lines, ['Greedy', 'Tsang et al.', 'Emb+K-med', 'Preproc+Emb+K-med'], bbox_to_anchor=(-0.24, 1.06, 1.4, .102), loc='lower left',
#            ncol=2, mode="expand", borderaxespad=0.)
#     plt.gca().add_artist(legend1)
    
    if ylim:
        ax.set_ylim([0, ylim])
    
    ax.set_axisbelow(True)
    ax.yaxis.grid(color='gray', linestyle='dashed')
    
    plt.legend(loc='upper left', prop={'size': legend_size})
    
    plt.xlabel('Number of Seeds', fontsize = labelsize)
#     plt.ylabel('Diff in Fraction Influenced', fontsize = labelsize)
    plt.ylabel('Discrepancy', fontsize = labelsize)
    plt.rcParams.update({'font.size': fontsize})
    plt.xticks(fontsize=labelsize)
    plt.yticks(fontsize=labelsize)
    fig.set_size_inches(imagesize[0], imagesize[1]) 
    
    added_s = ''
    if not square:
        added_s = '1'
    
    fig.savefig('fig/influence_maximization_rice/rice-diff' + added_s + '.pdf',bbox_inches='tight')


In [ ]:
def get_bar_plot_all(real, bar_width, square, fontsize, labelsize, imagesize, ylim = None):
    
    
    methods = ['Gr', 'Ts', 'EmKm', 'PrEmKm']
    
    #labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
    labels = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
    
#     xg= [xx-bar_width/2 , xx, xx+bar_width/2]
#     xp = [xx - bar_width/2, 3, 3 + bar_width/2]
#     xf = [xx - bar_width/2, 5, 5 + bar_width/2]
#     xfm = [xx - bar_width/2, 7, 7 + bar_width/2]
    
    
    
    xg = [1- bar_width, 1, 1 + bar_width]
    xa = [3 - bar_width, 3, 3 + bar_width]
    xe = [5 - bar_width, 5, 5 + bar_width]
    xp = [7 - bar_width, 7, 7 + bar_width]
    
    

    greedy = rice_greedy[-1, :3] # [0.16, 0.084,	0.18]    
    ad = adv[-1,:3]
    EmKm = rice_unweighted[-1, :3] # [0.161,	0.136, 0.168]           
    PrEmKm = rice_rw_method[-1, :3] # [0.175,	0.11, 0.194]       
    
    fig, ax = plt.subplots()
    
    ax.bar(xg[0], greedy[0], bar_width, color = red_, edgecolor='black')
    ax.bar(xa[0], ad[0], bar_width, color = cyan_, edgecolor='black')
    ax.bar(xe[0], EmKm[0], bar_width, color = green_, edgecolor='black')
    ax.bar(xp[0], PrEmKm[0], bar_width, color = blue_, edgecolor='black')
    
    ax.bar(xg[1], greedy[1], bar_width, color = red_, hatch='//', edgecolor='black')
    ax.bar(xa[1], ad[1], bar_width, color = cyan_, hatch='//', edgecolor='black')
    ax.bar(xe[1], EmKm[1], bar_width, color = green_, hatch='//', edgecolor='black')
    ax.bar(xp[1], PrEmKm[1], bar_width, color = blue_, hatch='//', edgecolor='black')
        
    ax.bar(xg[2], greedy[2], bar_width, color = red_, hatch='\\\\', edgecolor='black')
    ax.bar(xa[2], ad[2], bar_width, color = cyan_, hatch='\\\\', edgecolor='black')
    ax.bar(xe[2], EmKm[2], bar_width, color = green_, hatch='\\\\', edgecolor='black')
    ax.bar(xp[2], PrEmKm[2], bar_width, color = blue_, hatch='\\\\', edgecolor='black')
    
    ax.bar(xg[0], 0, bar_width, color = gray_, label='Total', edgecolor='black')
    ax.bar(xg[1], 0, bar_width, color = gray_, hatch='//', label='Group A', edgecolor='black')
    ax.bar(xg[2], 0, bar_width, color = gray_, hatch='\\\\', label='Group B', edgecolor='black')

    plt.legend(loc='upper left')
    
#     custom_lines = [Line2D([0], [0], color='red', lw=3),
#                     Line2D([0], [0], color='green', lw=3),
#                     Line2D([0], [0], color='blue', lw=3), 
#                     Line2D([0], [0], color='cyan', lw=3)]
#     legend1 = plt.legend(custom_lines, ['Greedy', 'Tsang et al.', 'Emb+K-med', 'Preproc+Emb+K-med'], bbox_to_anchor=(-0.24, 1.06, 1.4, .102), loc='lower left',
#                ncol=2, mode="expand", borderaxespad=0.)
#     plt.gca().add_artist(legend1)
    
#     custom_lines2 = [Line2D([0], [0], linestyle='dashed', lw=3),
#                     Line2D([0], [0], linestyle='dotted', lw=3)]
#     legend2 = plt.legend(custom_lines2, ['Group A', 'Group B'])
#     plt.gca().add_artist(legend2)
    
    if ylim:
        ax.set_ylim([0, ylim])
    
    ax.set_xticklabels(labels)
#     plt.xticks([0.6,1,1.4,2.6,3,3.4,4.6,5,5.4,6.6,7,7.4], ['Total','Group A','Group B','Total','Group A','Group B','Total','Group A','Group B','Total','Group A','Group B'], rotation=60, horizontalalignment='right', fontsize=12)
#     ax.set_axisbelow(True)
#     ax.yaxis.grid(color='gray', linestyle='dashed')
    
    plt.xlabel('Methods', fontsize = labelsize)
    plt.ylabel('Fraction Influenced', fontsize = labelsize)
    plt.rcParams.update({'font.size': fontsize})
    plt.xticks(fontsize=17)
    plt.yticks(fontsize=labelsize)
    fig.set_size_inches(imagesize[0], imagesize[1]) 
    
    added_s = ''
    if not square:
        added_s = '1'
    
    fig.savefig('fig/influence_maximization_rice/rice-bar' + added_s + '.pdf',bbox_inches='tight')
    

In [ ]:
def get_bar_plot(real, bar_width, square, fontsize, labelsize, imagesize, ylim = None, legend_size=20):
    
    
#     methods = ['Gr', 'Ts', 'EmKm', 'PrEmKm']
    
    #labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
    labels = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
    
#     xg= [xx-bar_width/2 , xx, xx+bar_width/2]
#     xp = [xx - bar_width/2, 3, 3 + bar_width/2]
#     xf = [xx - bar_width/2, 5, 5 + bar_width/2]
#     xfm = [xx - bar_width/2, 7, 7 + bar_width/2]
    
    
    
#     xu = [2 - bar_width, 2]
#     xf = [7.65 - bar_width, 7.65]
#     xa = [13.30 - bar_width, 13.30]
#     xp = [19 - bar_width, 19]

    xu = [2 - bar_width, 2]
    xf = [4 - bar_width, 4]
    xa = [6 - bar_width, 6]
    xp = [8 - bar_width, 8]
    
    

#     greedy = rice_greedy[-1, :3] # [0.16, 0.084,	0.18]    
    ad = adv[-1,:4]
    EmKm = rice_unweighted[-1, :4] # [0.161,	0.136, 0.168]
    FW = fairwalk[-1,:4]
    PrEmKm = rice_rw_method[-1, :4] # [0.175,	0.11, 0.194]       
    
    fig, ax = plt.subplots()
    
    ax.bar(xu[0], EmKm[0], bar_width, color = purple_, edgecolor='black', label='Total Influence Percentage')
    ax.bar(xf[0], FW[0], bar_width, color = purple_, edgecolor='black')
    ax.bar(xa[0], ad[0], bar_width, color = purple_, edgecolor='black')
    ax.bar(xp[0], PrEmKm[0], bar_width, color = purple_, edgecolor='black')
    
    ax.bar(xu[1], EmKm[3], bar_width, color = yellow_, edgecolor='black', label='Disparity')
    ax.bar(xf[1], FW[3], bar_width, color = yellow_, edgecolor='black')
    ax.bar(xa[1], ad[3], bar_width, color = yellow_, edgecolor='black')
    ax.bar(xp[1], PrEmKm[3], bar_width, color = yellow_, edgecolor='black')
    
#     ax.bar(xg[1], greedy[1], bar_width, color = red_, hatch='//', edgecolor='black')
#     ax.bar(xa[1], ad[1], bar_width, color = cyan_, hatch='//', edgecolor='black')
#     ax.bar(xe[1], EmKm[1], bar_width, color = green_, hatch='//', edgecolor='black')
#     ax.bar(xp[1], PrEmKm[1], bar_width, color = blue_, hatch='//', edgecolor='black')
        
#     ax.bar(xg[2], greedy[2], bar_width, color = red_, hatch='\\\\', edgecolor='black')
#     ax.bar(xa[2], ad[2], bar_width, color = cyan_, hatch='\\\\', edgecolor='black')
#     ax.bar(xe[2], EmKm[2], bar_width, color = green_, hatch='\\\\', edgecolor='black')
#     ax.bar(xp[2], PrEmKm[2], bar_width, color = blue_, hatch='\\\\', edgecolor='black')
    
#     ax.bar(xg[0], 0, bar_width, color = gray_, label='Total', edgecolor='black')
#     ax.bar(xg[1], 0, bar_width, color = gray_, hatch='//', label='Group A', edgecolor='black')
#     ax.bar(xg[2], 0, bar_width, color = gray_, hatch='\\\\', label='Group B', edgecolor='black')

#     plt.legend(loc='upper left')
    
#     custom_lines = [Line2D([0], [0], color='red', lw=3),
#                     Line2D([0], [0], color='green', lw=3),
#                     Line2D([0], [0], color='blue', lw=3), 
#                     Line2D([0], [0], color='cyan', lw=3)]
#     legend1 = plt.legend(custom_lines, ['Greedy', 'Tsang et al.', 'Emb+K-med', 'Preproc+Emb+K-med'], bbox_to_anchor=(-0.24, 1.06, 1.4, .102), loc='lower left',
#                ncol=2, mode="expand", borderaxespad=0.)
#     plt.gca().add_artist(legend1)
    
#     custom_lines2 = [Line2D([0], [0], linestyle='dashed', lw=3),
#                     Line2D([0], [0], linestyle='dotted', lw=3)]
#     legend2 = plt.legend(custom_lines2, ['Group A', 'Group B'])
#     plt.gca().add_artist(legend2)
    
    if ylim:
        ax.set_ylim([0, ylim])
    
    plt.legend(loc='upper right', prop={'size': legend_size})
    
    ax.set_xticklabels(labels)
    plt.xticks([2, 4, 6, 8], ['DeepWalk', 'FairWalk', 'Adversarial', 'CrossWalk'], fontsize=legend_size)
#     plt.xticks([0.6,1,1.4,2.6,3,3.4,4.6,5,5.4,6.6,7,7.4], ['Total','Group A','Group B','Total','Group A','Group B','Total','Group A','Group B','Total','Group A','Group B'], rotation=60, horizontalalignment='right', fontsize=12)
#     ax.set_axisbelow(True)
    ax.yaxis.grid(color='gray', linestyle='dashed')
    
#     plt.xlabel('Methods', fontsize = labelsize)
#     plt.ylabel('Fraction Influenced', fontsize = labelsize)
    plt.rcParams.update({'font.size': fontsize})
    plt.yticks(fontsize=labelsize)
    fig.set_size_inches(imagesize[0], imagesize[1]) 
    
    added_s = ''
    if not square:
        added_s = '1'
    
    fig.savefig('fig/influence_maximization_rice/rice-bar' + added_s + '.pdf',bbox_inches='tight')
    

In [ ]:
def get_mmd_plot(real, bar_width, fontsize, labelsize, imagesize, ylim, legend_size):
    
    #labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
    #labels = ['A-A', 'B-B', 'A-B', 'Total']
    
    xu = [ 2 - bar_width, 2, 2 + bar_width]
    x0_3 = [6 - bar_width, 6, 6 + bar_width]
    x0_5 = [10 - bar_width, 10, 10 + bar_width]
    x0_7 = [14 - bar_width, 14, 14 + bar_width]
    xr = [18 - bar_width, 18, 18 + bar_width]
    
    
    fig, ax = plt.subplots()
    
    ax.bar(xu[0], 100 * uw[0], bar_width, color = green_, edgecolor='black', label='MMD ✕ 100')
    ax.bar(x0_3[0], 100 * alpha0_3_p_4[0], bar_width, color = green_, edgecolor='black')
    ax.bar(x0_5[0], 100 * alpha0_5_p_4[0], bar_width, color = green_, edgecolor='black')
    ax.bar(x0_7[0], 100 * alpha0_7_p_4[0], bar_width, color = green_, edgecolor='black')
    ax.bar(xr[0], 100 * rnd[0], bar_width, color = green_, edgecolor='black')

    
    ax.bar(xu[1], uw[1], bar_width, color = purple_, edgecolor='black', label='Total Accuracy')
    ax.bar(x0_3[1], alpha0_3_p_4[1], bar_width, color = purple_, edgecolor='black')
    ax.bar(x0_5[1], alpha0_5_p_4[1], bar_width, color = purple_, edgecolor='black')
    ax.bar(x0_7[1], alpha0_7_p_4[1], bar_width, color = purple_, edgecolor='black')
    ax.bar(xr[1], rnd[1], bar_width, color = purple_, edgecolor='black')

    ax.bar(xu[2], uw[2], bar_width, color = yellow_, edgecolor='black', label='Disparity')
    ax.bar(x0_3[2], alpha0_3_p_4[2], bar_width, color = yellow_, edgecolor='black')
    ax.bar(x0_5[2], alpha0_5_p_4[2], bar_width, color = yellow_, edgecolor='black')
    ax.bar(x0_7[2], alpha0_7_p_4[2], bar_width, color = yellow_, edgecolor='black')
    ax.bar(xr[2], rnd[2], bar_width, color = yellow_, edgecolor='black')

    plt.legend(loc='upper right', prop={'size': legend_size}) #'upper left')
    
    ax.set_ylim(ylim)
     
    plt.xticks([2, 6, 10, 14, 18], ['DeepWalk', 'CrossWalk\nα=0.3, p=4', 'CrossWalk\nα=0.5, p=4', 'CrossWalk\nα=0.7, p=4', 'Random\nEmbedding'], fontsize = 18)
    ax.set_axisbelow(True)
    ax.yaxis.grid(color='gray', linestyle='dashed')
    
#     plt.xlabel('Methods', fontsize = labelsize)
#     plt.ylabel('Accuracy', fontsize = labelsize)
    plt.rcParams.update({'font.size': fontsize})
    plt.yticks(fontsize=labelsize)
    fig.set_size_inches(imagesize[0], imagesize[1]) 
    fig.savefig('fig/influence_maximization_rice/rice-mmd' + '.pdf',bbox_inches='tight')
    

In [ ]:
def get_legend():
    params = {'legend.fontsize': 12,
          'legend.handlelength': 2}
    plt.rcParams.update(params)
    fig, ax = plt.subplots(figsize=(8,.015))
    custom_lines = [Line2D([0], [0], color=red_, lw=3),
            Line2D([0], [0], color='cyan', lw=3),
            Line2D([0], [0], color='green', lw=3), 
            Line2D([0], [0], color='blue', lw=3)]
    legend1 = plt.legend(custom_lines, ['Greedy', 'Adversarial Embedding', 'Deepwalk + K-medoids', 'Reweighting + Deepwalk + K-medoids'], bbox_to_anchor=(-0.24, 1.06, 1.4, .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
    plt.gca().add_artist(legend1)
    plt.axis('off')
    fig.savefig('fig/influence_maximization_rice/rice-legend.pdf', bbox_inches='tight')

In [ ]:
# Plotting the square plots
# Real Plots

label_size = 27
font_size = 24
image_size = (12, 8.5)
line_width = 3
square = True
bar_width = 0.5
# y_lim = 0.3

get_real_plots(figtype='total', square=square, fontsize=font_size, labelsize=label_size, imagesize=image_size, line_width=line_width, ylim=20, legend_size = 20)
get_real_plots(figtype='frac', square=square, fontsize=font_size, labelsize=label_size, imagesize=image_size, line_width=line_width, ylim=23, legend_size = 20)
# get_diff_plot(real=True, square=square, fontsize=font_size, labelsize=label_size, imagesize=image_size, line_width=line_width, ylim=0.16)
get_var_plot(real=True, square=square, fontsize=font_size, labelsize=label_size, imagesize=image_size, line_width=line_width, ylim=50, legend_size=20)
get_bar_plot(real=True, bar_width=bar_width, square=square, fontsize=font_size, labelsize=label_size, imagesize=image_size, ylim=50, legend_size=25)
get_legend()

image_size = (12, 8.5)
y_lim = [0, 70]
legend_size = 20
get_mmd_plot(real=True, bar_width=bar_width, fontsize=font_size, labelsize=label_size, imagesize=image_size, ylim=y_lim, legend_size=legend_size)